# Clone & Compare Workflow

This notebook demonstrates the **CLB Engineering LLM Forward Approach** to QAQC:
- Non-destructive cloning of basin, met, and run configurations
- Side-by-side comparison in HEC-HMS GUI
- Parallel execution of baseline vs. updated scenarios

## Why Clone?

Cloning allows you to:
1. Preserve original baseline models
2. Test modifications without risk
3. Compare results in GUI side-by-side
4. Maintain traceable metadata (descriptions updated with clone info)
5. Use separate DSS files for clean comparison

In [ ]:
# Flexible imports for development
from pathlib import Path
import sys

try:
    from hms_commander import (
        init_hms_project, hms,
        HmsBasin, HmsMet, HmsRun, HmsCmdr, HmsResults
    )
except ImportError:
    sys.path.append(str(Path().resolve().parent))
    from hms_commander import (
        init_hms_project, hms,
        HmsBasin, HmsMet, HmsRun, HmsCmdr, HmsResults
    )

## 1. Initialize Project and Extract Example

In [ ]:
from hms_commander import HmsExamples

# Extract HMS example project
project_path = HmsExamples.extract_project(
    "castro",
    output_path=Path.cwd() / 'example_projects' / 'castro'
)
print(f"Project extracted to: {project_path}")

# Initialize project
init_hms_project(project_path)

# View available components
print("\nBasin Models:")
print(hms.basin_df)
print("\nMet Models:")
print(hms.met_df)
print("\nRuns:")
print(hms.run_df)

## 2. Clone Basin Model

In [ ]:
# Clone basin for modifications
success = HmsBasin.clone_basin(
    template="Castro",
    new_name="Castro_Updated",
    description="Updated CN values for improved calibration"
)

if success:
    print("Basin cloned successfully!")
    print("New basin will appear in HEC-HMS GUI")

## 3. Modify Parameters in Cloned Basin

In [ ]:
# Get basin file path
basin_file = project_path / "Castro_Updated.basin"

# Get all subbasins
subbasins = HmsBasin.get_subbasins(str(basin_file))
print(f"Found {len(subbasins)} subbasins")

# Update curve numbers for all subbasins
for idx, row in subbasins.iterrows():
    subbasin_name = row['name']
    
    # Get current parameters
    params = HmsBasin.get_loss_parameters(str(basin_file), subbasin_name)
    
    if 'curve_number' in params:
        old_cn = params['curve_number']
        new_cn = old_cn + 5  # Increase CN by 5
        
        # Update
        HmsBasin.set_loss_parameters(
            str(basin_file),
            subbasin_name,
            curve_number=new_cn
        )
        print(f"{subbasin_name}: CN {old_cn} → {new_cn}")

## 4. Clone Meteorologic Model (if needed)

In [ ]:
# Clone met model if you need to modify precipitation
success = HmsMet.clone_met(
    template="Castro",
    new_name="Castro_Updated",
    description="Updated precipitation depths"
)

print(f"Met model cloned: {success}")

## 5. Clone Run Configuration

**Critical for QAQC**: Separate DSS output files for baseline vs. updated

In [ ]:
# Clone run with updated components and separate DSS output
success = HmsRun.clone_run(
    source_run="Run 1",
    new_run_name="Run 1 - Updated",
    new_basin="Castro_Updated",
    new_met="Castro_Updated",  # Optional, use if you cloned met
    output_dss="castro_updated.dss",
    description="Updated CN values (+5)"
)

if success:
    print("Run configuration cloned!")
    print("Baseline: Run 1 → castro.dss")
    print("Updated:  Run 1 - Updated → castro_updated.dss")

## 6. Configure Cloned Runs (NEW)

After cloning runs, configure them with updated descriptions and verify component assignments. 

**Phase 1 Feature**: Use the new `HmsRun.set_*()` methods to modify run configurations safely.

In [ ]:
from hms_commander import HmsRun

# Update the cloned run's description for QAQC tracking
HmsRun.set_description(
    run_name="Clone_Run",  # Use your cloned run name
    description="Cloned configuration for QAQC comparison - Phase 1 demonstration",
    hms_object=hms
)
print("[OK] Updated cloned run description")

# Set separate DSS output file for comparison
HmsRun.set_dss_file(
    run_name="Clone_Run",
    dss_file="clone_run_results.dss",  # Separate file for side-by-side comparison
    hms_object=hms
)
print("[OK] Set separate DSS output for comparison")

### Verify Both Run Configurations

Display baseline and cloned run configurations side-by-side:

In [ ]:
# Reinitialize to refresh DataFrames
from hms_commander import init_hms_project
init_hms_project(hms.project_folder, hms_object=hms)

# Get baseline run configuration
print("Baseline Run Configuration:")
print("=" * 60)
baseline_runs = [r for r in hms.list_run_names() if 'Clone' not in r]
if baseline_runs:
    baseline_config = HmsRun.get_dss_config(baseline_runs[0], hms_object=hms)
    print(f"  Run Name:   {baseline_runs[0]}")
    print(f"  Basin:      {baseline_config['basin_model']}")
    print(f"  Met:        {baseline_config['met_model']}")
    print(f"  Control:    {baseline_config['control_spec']}")
    print(f"  DSS Output: {baseline_config['dss_file']}")

# Get cloned run configuration
print("\nCloned Run Configuration:")
print("=" * 60)
cloned_runs = [r for r in hms.list_run_names() if 'Clone' in r]
if cloned_runs:
    cloned_config = HmsRun.get_dss_config(cloned_runs[0], hms_object=hms)
    print(f"  Run Name:   {cloned_runs[0]}")
    print(f"  Basin:      {cloned_config['basin_model']}")
    print(f"  Met:        {cloned_config['met_model']}")
    print(f"  Control:    {cloned_config['control_spec']}")
    print(f"  DSS Output: {cloned_config['dss_file']}")
    print(f"  Description: {cloned_config.get('description', 'N/A')}")

### Why Separate DSS Files Matter

Using different DSS output files for baseline vs. cloned runs enables:

1. **Side-by-side comparison** - Both result sets available simultaneously
2. **No data overwriting** - Baseline results preserved during testing
3. **QAQC workflow** - Compare results before committing to new configuration
4. **Documentation** - Clear separation of baseline vs. updated scenarios

This is the **CLB Engineering LLM Forward** approach: non-destructive, traceable, GUI-verifiable.

## 7. Execute Both Runs in Parallel

In [ ]:
# Run baseline and updated scenarios in parallel
results = HmsCmdr.compute_parallel(
    run_names=["Run 1", "Run 1 - Updated"],
    max_workers=2
)

for run_name, success in results.items():
    status = "✓ Success" if success else "✗ Failed"
    print(f"{run_name}: {status}")

## 8. Compare Results

In [ ]:
# Compare peak flows
baseline_dss = project_path / "castro.dss"
updated_dss = project_path / "castro_updated.dss"

if baseline_dss.exists() and updated_dss.exists():
    # Get peak flows from both runs
    baseline_peaks = HmsResults.get_peak_flows(str(baseline_dss))
    updated_peaks = HmsResults.get_peak_flows(str(updated_dss))
    
    print("\n=== Peak Flow Comparison ===")
    print("\nBaseline:")
    print(baseline_peaks)
    print("\nUpdated:")
    print(updated_peaks)
    
    # Calculate differences
    import pandas as pd
    comparison = pd.DataFrame({
        'Element': baseline_peaks['element'],
        'Baseline_cfs': baseline_peaks['peak_flow_cfs'],
        'Updated_cfs': updated_peaks['peak_flow_cfs'],
    })
    comparison['Difference_cfs'] = comparison['Updated_cfs'] - comparison['Baseline_cfs']
    comparison['Percent_Change'] = (
        (comparison['Updated_cfs'] - comparison['Baseline_cfs']) / 
        comparison['Baseline_cfs'] * 100
    )
    
    print("\n=== Comparison Summary ===")
    print(comparison)
else:
    print("\nRun simulations first to generate DSS files")

## 9. Visualize Comparison

In [ ]:
import matplotlib.pyplot as plt

# Get hydrographs for outlet
outlet_name = "Outlet"  # Adjust to your model

baseline_ts = HmsResults.get_outflow_timeseries(str(baseline_dss), outlet_name)
updated_ts = HmsResults.get_outflow_timeseries(str(updated_dss), outlet_name)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(baseline_ts.index, baseline_ts.values, label='Baseline', linewidth=2)
plt.plot(updated_ts.index, updated_ts.values, label='Updated CN (+5)', 
         linewidth=2, linestyle='--')
plt.xlabel('Time')
plt.ylabel('Flow (cfs)')
plt.title(f'{outlet_name} Hydrograph Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Verify in HEC-HMS GUI

After running this notebook:

1. Open the project in HEC-HMS GUI
2. You should see:
   - Original basin: `Castro`
   - Cloned basin: `Castro_Updated`
   - Original run: `Run 1`
   - Cloned run: `Run 1 - Updated`
3. Open both result plots side-by-side for visual comparison
4. Check descriptions for clone metadata

## Summary

This workflow demonstrates:

✓ **Non-destructive**: Original models preserved  
✓ **Traceable**: Clone metadata in descriptions  
✓ **GUI-verifiable**: Components visible in HEC-HMS  
✓ **Efficient**: Parallel execution  
✓ **Clean comparison**: Separate DSS outputs  

Perfect for:
- Parameter sensitivity analysis
- Model updates and calibration
- QAQC workflows
- Scenario comparison
